# Feedforward Neural Network

Feedforward refers to recognition-inference architecture of neural networks. Artificial neural network architectures are based on inputs multiplied by weights to obtain outputs (inputs-to-output): feedforward.Recurrent neural networks, or neural networks with loops allow information from later processing stages to feed back to earlier stages for sequence processing. However, at every stage of inference a feedforward multiplication remains the core, essential for backpropagation or backpropagation through time. Thus neural networks cannot contain feedback like negative feedback or positive feedback where the outputs feed back to the very same inputs and modify them, because this forms an infinite loop which is not possible to rewind in time to generate an error signal through backpropagation. This issue and nomenclature appear to be a point of confusion between some computer scientists and scientists in other fields studying brain networks.

## Simple Form

In [1]:
# Import Libraries

import numpy as np

In [2]:
# Define the Activation Function

def sigmoid(x):
    return 1 / (1 + np.exp(-x))

# Define the Derivative of the Activation Function

def sigmoid_derivative(x):
    return x * (1 - x)

In [3]:
# Create Dummy input and Output Data

# Input: 4 samples with 2 features each
X = np.array([[0,0],
              [0,1],
              [1,0],
              [1,1]])

# Expected output for XOR problem
y = np.array([[0],
              [1],
              [1],
              [0]])

In [4]:
# Initialize Weights and Biases

np.random.seed(42)  # For reproducibility

input_neurons = X.shape[1]         # Number of input features
hidden_neurons = 4                  # Number of neurons in the hidden layer
output_neurons = 1                 # Number of output neurons

# Weights between input and hidden layer
W1 = np.random.rand(input_neurons, hidden_neurons)
# Bias for hidden layer
B1 = np.random.rand(1, hidden_neurons)

# Weights between hidden and output layer
W2 = np.random.rand(hidden_neurons, output_neurons)
# Bias for output layer
B2 = np.random.rand(1, output_neurons)

In [5]:
# Training Loop (Forward and Backward Propagation)

learning_rate = 0.1
epochs = 10000

for epoch in range(epochs):
    # ---Forward Propagation---
    Z1 = np.dot(X, W1) + B1  # Weighted sum for hidden layer
    A1 = sigmoid(Z1)         # Activation for hidden layer

    Z2 = np.dot(A1, W2) + B2  # Weighted sum for output layer
    A2 = sigmoid(Z2)          # Activation for output layer

    # ---Backward Propagation---
    error = y - A2  # Calculate error
    dA2 = error * sigmoid_derivative(A2)  # Derivative of output layer

    error_hidden = dA2.dot(W2.T)  # Backpropagation to hidden layer
    dA1 = error_hidden * sigmoid_derivative(A1)  # Derivative of hidden layer

    # ---Update Weights and Biases---
    W2 += A1.T.dot(dA2) * learning_rate  # Update weights for output layer
    B2 += np.sum(dA2, axis=0, keepdims=True) * learning_rate  # Update bias for output layer

    W1 += X.T.dot(dA1) * learning_rate  # Update weights for hidden layer
    B1 += np.sum(dA1, axis=0, keepdims=True) * learning_rate  # Update bias for hidden layer

    # ---Print loss occasionally---
    if epoch % 1000 == 0:
        loss = np.mean(np.square(error))  # Mean Squared Error
        print(f'Epoch {epoch}, Loss: {loss:.4f}')

Epoch 0, Loss: 0.3322
Epoch 1000, Loss: 0.2490
Epoch 2000, Loss: 0.2444
Epoch 3000, Loss: 0.2180
Epoch 4000, Loss: 0.1561
Epoch 5000, Loss: 0.0560
Epoch 6000, Loss: 0.0209
Epoch 7000, Loss: 0.0109
Epoch 8000, Loss: 0.0069
Epoch 9000, Loss: 0.0049


In [6]:
# Test the Output

print("Final Output after Training:")
print(A2.round(2))  # Output after training

Final Output after Training:
[[0.05]
 [0.96]
 [0.93]
 [0.07]]


## PyTorch Form

In [7]:
# Import Libraries

import torch
import torch.nn as nn
import torch.optim as optim

In [8]:
# Input: 4 samples with 2 features
X = torch.tensor([[0,0],
                [0,1],
                [1,0],
                [1,1]], dtype = torch.float32)

# Target output
y = torch.tensor([[0],
                [1],
                [1],
                [0]], dtype = torch.float32)

In [9]:
# Define the Neural Network Model

class FFNN(nn.Module):
    def __init__(self):
        super (FFNN, self).__init__()
        self.fc1 = nn.Linear(2, 4)      # 2 input features, 4 hidden neurons
        self.fc2 = nn.Linear(4, 1)      # 4 hidden neurons, 1 output neuron

    def forward(self, x):
        x = torch.sigmoid(self.fc1(x))  # Hidden layer activation
        x = torch.sigmoid(self.fc2(x))  # Output layer activation
        return x
    

In [10]:
# Create Model, Loss Function and Optimizer

model = FFNN()

# Binary Cross Entropy is suitable for binary classification
criterion = nn.BCELoss()

# Stochastic Gradient Descent optimizer
optimizer = optim.SGD(model.parameters(), lr=0.1)

In [11]:
# Training Loop

epochs = 10000

for epoch in range(epochs):

    # ---Forward Pass---
    outputs = model(X)  # Forward pass
    loss = criterion(outputs, y)  # Calculate loss

    # ---Backward Pass---
    optimizer.zero_grad()  # Zero the gradients
    loss.backward()  # Backpropagation
    optimizer.step()  # Update weights

    # ---Print loss occasionally---
    if epoch % 1000 == 0:
        print(f'Epoch {epoch}, Loss: {loss.item():.4f}')

C:\Users\tchibana\AppData\Local\Temp\ipykernel_635732\3067645477.py:18: UserWarning: Converting a tensor with requires_grad=True to a scalar may lead to unexpected behavior.
Consider using tensor.detach() first. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\pytorch\aten\src\ATen\native\Scalar.cpp:23.)
  print(f'Epoch {epoch}, Loss: {loss.item():.4f}')


Epoch 0, Loss: 0.7006
Epoch 1000, Loss: 0.6920
Epoch 2000, Loss: 0.6867
Epoch 3000, Loss: 0.6283
Epoch 4000, Loss: 0.3510
Epoch 5000, Loss: 0.1201
Epoch 6000, Loss: 0.0594
Epoch 7000, Loss: 0.0375
Epoch 8000, Loss: 0.0268
Epoch 9000, Loss: 0.0206


In [12]:
# Test the Model

with torch.no_grad(): # no gradient needed
    predictions = model(X)  # Forward pass
    print("Predicted Output:")
    print(predictions.round())  # Round the output to get binary predictions
    

Predicted Output:
tensor([[0.],
        [1.],
        [1.],
        [0.]])


In [14]:
print(torch.cuda.is_available())  # Check if CUDA is available
print(torch.version.cuda)

True
12.8


## Pytorch With GPU

In [16]:
# Import Libraries

import torch
import torch.nn as nn
import torch.optim as optim

In [17]:
# Select the device: GPU if available, else CPU

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")  # Print the selected device

Using device: cuda


In [18]:
# Input: 4 samples with 2 features
X = torch.tensor([[0,0],
                [0,1],
                [1,0],
                [1,1]], dtype = torch.float32).to(device) # Move to device

# Target output
y = torch.tensor([[0],
                [1],
                [1],
                [0]], dtype = torch.float32).to(device) # Move to device

In [ ]:
# Define the Neural Network Model

class FFNN(nn.Module):
    def __init__(self):
        super (FFNN, self).__init__()
        self.fc1 = nn.Linear(2, 4)      # 2 input features, 4 hidden neurons
        self.fc2 = nn.Linear(4, 1)      # 4 hidden neurons, 1 output neuron

    def forward(self, x):
        x = torch.sigmoid(self.fc1(x))  # Hidden layer activation
        x = torch.sigmoid(self.fc2(x))  # Output layer activation
        return x
    

In [19]:
# Create Model, Loss Function and Optimizer

model = FFNN().to(device)  # Move model to device

# Binary Cross Entropy is suitable for binary classification
criterion = nn.BCELoss()

# Stochastic Gradient Descent optimizer
optimizer = optim.SGD(model.parameters(), lr=0.1)

In [20]:
# Training Loop

epochs = 10000

for epoch in range(epochs):

    # ---Forward Pass---
    outputs = model(X)  # Forward pass
    loss = criterion(outputs, y)  # Calculate loss

    # ---Backward Pass---
    optimizer.zero_grad()  # Zero the gradients
    loss.backward()  # Backpropagation
    optimizer.step()  # Update weights

    # ---Print loss occasionally---
    if epoch % 1000 == 0:
        print(f'Epoch {epoch}, Loss: {loss.item():.4f}')

Epoch 0, Loss: 0.7079
Epoch 1000, Loss: 0.6932
Epoch 2000, Loss: 0.6930
Epoch 3000, Loss: 0.6928
Epoch 4000, Loss: 0.6918
Epoch 5000, Loss: 0.6853
Epoch 6000, Loss: 0.6293
Epoch 7000, Loss: 0.4138
Epoch 8000, Loss: 0.1478
Epoch 9000, Loss: 0.0680


In [22]:
# Test the Model

with torch.no_grad(): # no gradient needed
    predictions = model(X)  # Forward pass
    print("Predicted Output:")
    print(predictions.round().cpu())  # Round the output to get binary predictions and move to CPU for printing
    

Predicted Output:
tensor([[0.],
        [1.],
        [1.],
        [0.]])
